In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
img1 = cv2.imread('3.jpg', cv2.IMREAD_GRAYSCALE)  
img2 = cv2.imread('4.jpg', cv2.IMREAD_GRAYSCALE)  

In [ ]:
img1_cpy=cv2.imread('3.jpg', cv2.IMREAD_COLOR);
img2_cpy=cv2.imread('4.jpg', cv2.IMREAD_COLOR);

In [ ]:
if img1 is None or img2 is None:
    print("fail")
    exit()

In [ ]:
t1 = time.time()
time_sum=0;
print(1)

In [ ]:
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(img1, None)
kp2, des2 = orb.detectAndCompute(img2, None)
print(2);

In [ ]:
des1 = np.float32(des1)
des2 = np.float32(des2)

In [ ]:
index_params = dict(algorithm=1, trees=3)  
search_params = dict(checks=10)  
flann = cv2.FlannBasedMatcher(index_params, search_params)

In [ ]:
matches = flann.knnMatch(des1, des2, k=2)

In [ ]:
print(3)

In [ ]:
good_matches = []
ratio_thresh = 0.75  
for m, n in matches:
    if m.distance < ratio_thresh * n.distance:
        good_matches.append(m)

In [ ]:
print(4)

In [ ]:
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
t2=time.time();
time_sum+=t2-t1;

In [ ]:
plt.figure(figsize=(10, 5))
plt.imshow(img_matches)
plt.title("Good Matches with Ratio Test")
plt.show()

In [ ]:
t1=time.time();
print('final')

In [ ]:
if len(good_matches) > 4:

    src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)


    H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)



    h1, w1 = img1.shape
    h2, w2 = img2.shape
    img1=img1_cpy
    img2=img2_cpy
    result = cv2.warpPerspective(img1, H, (w1 + w2, h1))
    result[0:h2, 0:w2] = img2
    t2=time.time();
    time_sum+=t2-t1;
    print('over')

    cv2.imwrite('34.jpg',result);
    plt.figure(figsize=(10, 5))
    plt.imshow(result, cmap='gray')
    plt.title("Image Stitching Result")
    plt.show()
else:
    print("no matrix")
    exit();

In [ ]:
elapsed_time = time_sum;
fps = 1 / elapsed_time if elapsed_time > 0 else 0

In [ ]:
print("fps:",fps);
print("time_cost(second):",time_sum);